In [3]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Train Model

In [4]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [5]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))

Files already downloaded
3000
750


In [6]:
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [32]:
def get_percentage(x):
    y = x.data.numpy()
    return len(y[np.where(y > 0)])*100./ y.size


In [33]:
class SimpleNet(nn.Module):
    def __init__(self):
        self.count = 0
        super(SimpleNet, self).__init__()
        self.linear1 = nn.Linear(784, 20)
        self.linear2 = nn.Linear(20, 10)

    def forward(self,x ):
        x = x.view(-1, 784)
        x = F.dropout(F.relu(self.linear1(x)), p = 0.3)
        print("Layer 1", get_percentage(x))
        x = F.dropout(F.relu(self.linear2(x)), p = 0.7)
        print("Layer 2", get_percentage(x))

#         if self.count < 3:
#             print(x[0])
        self.count += 1
        return F.log_softmax(x)
    
        

In [34]:
# CPU only training


def train(epoch):
    model.train()
    train_loss = 0
    layer_1_weights = 0
    layer_2_weights = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        train_loss += F.nll_loss(output, target).data[0]
        loss.backward()
        optimizer.step()
#         if batch_idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.data[0]))
    layer_1_weights = model.state_dict()['linear1.weight']
    layer_2_weights = model.state_dict()['linear2.weight']
        
    
    train_loss /= len(train_loader)
    
    return train_loss, layer_1_weights, layer_2_weights
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    layer_1_weights = 0
    layer_2_weights = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
    layer_1_weights = model.state_dict()['linear1.weight']
    layer_2_weights = model.state_dict()['linear2.weight']

    test_loss /= len(valid_loader) # loss function already averages over batch size
#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(valid_loader.dataset),
#         100. * correct / len(valid_loader.dataset)))
    return test_loss, layer_1_weights, layer_2_weights

In [35]:
train_errors = []
test_errors = []
train_layer_1 = []
test_layer_1 = []
train_layer_2 = []
test_layer_2 = []
model = SimpleNet()
optimizer = optim.SGD(model.parameters(), lr=0.01)


for epoch in range(1, 2):
    print("Train outputs")

    trainerror, trlayer_1, trlayer_2 = train(epoch)
    print("Test outputs")

    testerror, tslayer_1, tslayer_2 = test(epoch, valid_loader)




Train outputs
Layer 1 41.796875
Layer 2 39.53125
Layer 1 41.25
Layer 2 41.875
Layer 1 39.84375
Layer 2 39.53125
Layer 1 38.90625
Layer 2 40.625
Layer 1 37.8125
Layer 2 42.1875
Layer 1 37.65625
Layer 2 40.15625
Layer 1 39.921875
Layer 2 39.84375
Layer 1 38.59375
Layer 2 39.6875
Layer 1 37.578125
Layer 2 39.21875
Layer 1 37.03125
Layer 2 39.375
Layer 1 39.21875
Layer 2 38.4375
Layer 1 37.5
Layer 2 39.375
Layer 1 39.296875
Layer 2 41.875
Layer 1 38.984375
Layer 2 41.25
Layer 1 38.203125
Layer 2 44.0625
Layer 1 39.21875
Layer 2 42.8125
Layer 1 40.9375
Layer 2 39.375
Layer 1 39.609375
Layer 2 40.9375
Layer 1 39.21875
Layer 2 41.875
Layer 1 40.703125
Layer 2 43.90625
Layer 1 41.796875
Layer 2 41.71875
Layer 1 40.078125
Layer 2 41.25
Layer 1 38.359375
Layer 2 40.15625
Layer 1 41.5625
Layer 2 42.34375
Layer 1 41.953125
Layer 2 42.1875
Layer 1 41.015625
Layer 2 43.4375
Layer 1 41.40625
Layer 2 40.625
Layer 1 42.109375
Layer 2 39.84375
Layer 1 41.796875
Layer 2 42.1875
Layer 1 41.640625
Layer 2 

In [228]:
with_params = model.parameters()
for param, something in enumerate(with_params):
    print("Layer number ", param)
    print("Size of the layer", something.size())
    if param == 3:
        print(something)
    #print(something)


Layer number  0
Size of the layer torch.Size([20, 784])
Layer number  1
Size of the layer torch.Size([20])
Layer number  2
Size of the layer torch.Size([10, 20])
Layer number  3
Size of the layer torch.Size([10])
Parameter containing:
-0.0432
 0.0948
 0.0547
 0.1933
-0.0653
-0.1392
-0.1988
 0.0774
-0.1776
-0.2237
[torch.FloatTensor of size 10]



In [230]:
model.eval()


with_params = model.parameters()

model.count = 0
for data, target in valid_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)

for param, something in enumerate(with_params):
    if param == 3:
        print(something)  


Variable containing:
 0.0000
 0.0000
 0.2284
 0.0000
 3.5816
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.FloatTensor of size 10]

Variable containing:
 0.0000
 0.2822
 0.0835
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.FloatTensor of size 10]

Variable containing:
 0.0000
 3.0092
 0.6443
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.FloatTensor of size 10]

Parameter containing:
-0.0432
 0.0948
 0.0547
 0.1933
-0.0653
-0.1392
-0.1988
 0.0774
-0.1776
-0.2237
[torch.FloatTensor of size 10]

